Create dataset for downloaded videos

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import os
from EDA.eda_functions import get_label

In [2]:
ijmond_df = pd.read_json('../data/datasets/ijmond_dataset.json')
#predict label for each video in ijmond dir
files = os.listdir("../data/ijmond_videos/")

# Extract file names without extensions
file_names = [os.path.splitext(file)[0] for file in files]

# Filter the ijmond DataFrame to include only rows with matching file names
filtered_ijmond_df = ijmond_df[ijmond_df['file_name'].isin(file_names)]

In [3]:
filtered_ijmond_df['label'] = filtered_ijmond_df.apply(get_label, axis = 1)

/var/folders/9v/r3fdnxqn6v740_k7q9q14pym0000gn/T/ipykernel_15114/2910654267.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_ijmond_df['label'] = filtered_ijmond_df.apply(get_label, axis = 1)


In [4]:
filtered_ijmond_df['label'].value_counts()

1    13
0    13
Name: label, dtype: int64

In [5]:
#Save the created dataset
filtered_ijmond_df.to_json("../data/datasets/experimental_ijmond_dataset.json", orient='records')

Trying codes from CLIP documentation

In [6]:
import os
import clip
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import json
import cv2
from PIL import Image
from EDA.eda_functions import get_label

# Define device
if torch.cuda.is_available():
    device = torch.device("cuda") # use CUDA device
elif torch.backends.mps.is_available():
    device = torch.device("mps") # use MacOS GPU device (e.g., for M2 chips)
else:
    device = torch.device("cpu") # use CPU device
device

#Load the model
model, preprocess = clip.load('ViT-B/32', device)

#Define class names in a list - it needs prompt engineering
class_names = ["a photo of industrial plants emiting smoke from chimney", "a photo of industrial plants with no smoke above chimney"]

#Function to create an image from the video
def preprocess_video_to_image(video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    frames = []
    if not video.isOpened():
        print("Error: Could not open video file")
    else:
        while True:
            ret, frame = video.read()
            if not ret:
                break
            frames.append(frame)
        video.release()
    # Concatenate frames horizontally to create a single image
    concatenated_frame = np.concatenate(frames, axis=1)
    return concatenated_frame

#func to get the true label of the video
def get_true_label(file_name):
    row = ijmond_df[ijmond_df['file_name'] == file_name].iloc[0]
    return get_label(row)

#function to gwt label of the image produced from the video
def vanilla_clip(video_path):
    # Preprocess video into a single image
    video_image = preprocess_video_to_image(video_path)
    
    # Read image and preprocess
    image = preprocess(Image.fromarray(video_image)).unsqueeze(0).to(device)

    # Prepare text inputs based on class names list
    text_inputs = clip.tokenize(class_names).to(device)

    # Calculate features
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_inputs)

    # Calculate similarity
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    #values are the probabilities, indices are the classes
    values, indices = similarity[0].topk(2)

    # Print predictions
    print(f"\nPredictions for the entire video:\n")
    for value, index in zip(values, indices):
        print(f"{class_names[index]:>16s}: {100 * value.item():.2f}%")

#predict label for each video in ijmond dir
files = os.listdir("data/ijmond_videos/")
for file in files:
    video_path = f"data/ijmond_videos/{file}"
    file_name = file.split('.')[0]
    vanilla_clip(video_path)
    get_true_label(file_name)



/Users/szaboreka/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/szaboreka/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <85A36C65-3F71-3C3B-B529-961AE17DBE73> /Users/szaboreka/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <F8622D92-25A9-3A61-A089-C917FDA36C1B> /Users/szaboreka/anaconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(



Predictions for the entire video:

a photo of industrial plants with no smoke above chimney: 92.43%
a photo of industrial plants emiting smoke from chimney: 7.59%

Predictions for the entire video:

a photo of industrial plants with no smoke above chimney: 91.99%
a photo of industrial plants emiting smoke from chimney: 8.04%

Predictions for the entire video:

a photo of industrial plants with no smoke above chimney: 92.33%
a photo of industrial plants emiting smoke from chimney: 7.70%

Predictions for the entire video:

a photo of industrial plants with no smoke above chimney: 85.60%
a photo of industrial plants emiting smoke from chimney: 14.42%

Predictions for the entire video:

a photo of industrial plants with no smoke above chimney: 80.57%
a photo of industrial plants emiting smoke from chimney: 19.43%

Predictions for the entire video:

a photo of industrial plants with no smoke above chimney: 87.35%
a photo of industrial plants emiting smoke from chimney: 12.59%

Predictions 

In [8]:
import os
import clip
import torch
import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import json
import cv2
from PIL import Image
from EDA.eda_functions import get_label

# Define device
if torch.cuda.is_available():
    device = torch.device("cuda") # use CUDA device
elif torch.backends.mps.is_available():
    device = torch.device("mps") # use MacOS GPU device (e.g., for M2 chips)
else:
    device = torch.device("cpu") # use CPU device
device

#Load the model
model, preprocess = clip.load('ViT-B/32', device)

#Define class names in a list - it needs prompt engineering
class_names = ["a photo of industrial plants emiting smoke from chimney", "a photo of industrial plants with no smoke above chimney"]

#Function to create an image from the video
def preprocess_video_to_image_grid_version(video_path, num_rows=6, num_cols=6):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    frames = []
    if not video.isOpened():
        print("Error: Could not open video file")
    else:
        while True:
            ret, frame = video.read()
            if not ret:
                break
            frames.append(frame)
        video.release()
    
        # Split frames into grid
    grid = []
    for i in range(num_rows):
        row = np.concatenate(frames[i * num_cols: (i + 1) * num_cols], axis=1)
        grid.append(row)
    
    # Concatenate grid vertically to create a single image
    concatenated_frame = np.concatenate(grid, axis=0)
    return concatenated_frame

#func to get the true label of the video
def get_true_label(file_name):
    row = ijmond_df[ijmond_df['file_name'] == file_name].iloc[0]
    return get_label(row)

#function to gwt label of the image produced from the video
def vanilla_clip(video_path):
    # Preprocess video into a single image
    video_image = preprocess_video_to_image_grid_version(video_path)
    print(video_image.shape)
    # Convert numpy array to PIL Image
    #image = Image.fromarray(video_image)
    # Save the image
    #image.save('square_image.png')
    
    # Read image and preprocess
    image = preprocess(Image.fromarray(video_image)).unsqueeze(0).to(device)

    # Prepare text inputs based on class names list
    text_inputs = clip.tokenize(class_names).to(device)

    # Calculate features
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text_inputs)

    # Calculate similarity
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    #values are the probabilities, indices are the classes
    values, indices = similarity[0].topk(2)

    # Print predictions
    print(f"\nPredictions for the entire video {video_path}:\n")
    for value, index in zip(values, indices):
        print(f"{class_names[index]:>16s}: {100 * value.item():.2f}%")

#predict label for each video in ijmond dir
files = os.listdir("data/ijmond_videos/")
for file in files:
    video_path = f"data/ijmond_videos/{file}"
    file_name = file.split('.')[0]
    vanilla_clip(video_path)
    get_true_label(file_name)

#test:
#vanilla_clip('data/ijmond_videos/5PurGkmy0aw-1.mp4')


(3600, 3600, 3)

Predictions for the entire video data/ijmond_videos/A9W8G55JucU-3.mp4:

a photo of industrial plants with no smoke above chimney: 98.10%
a photo of industrial plants emiting smoke from chimney: 1.94%
(3600, 3600, 3)

Predictions for the entire video data/ijmond_videos/tT4vETXW7Og-2.mp4:

a photo of industrial plants with no smoke above chimney: 98.63%
a photo of industrial plants emiting smoke from chimney: 1.34%
(3600, 3600, 3)

Predictions for the entire video data/ijmond_videos/aINMnqmwSUg-1.mp4:

a photo of industrial plants with no smoke above chimney: 99.51%
a photo of industrial plants emiting smoke from chimney: 0.46%
(3600, 3600, 3)

Predictions for the entire video data/ijmond_videos/cxfcZFPpflE-0.mp4:

a photo of industrial plants with no smoke above chimney: 89.50%
a photo of industrial plants emiting smoke from chimney: 10.52%
(3600, 3600, 3)

Predictions for the entire video data/ijmond_videos/Qv9-nS5BloI-2.mp4:

a photo of industrial plants with no smoke